## Test C++ ElevenBand and Python bindings

Verify output levels = input levels.  Then adjust g50 and verify output levels have increased.

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import numpy as np
from mlab import generate_sine_waves
import plotly.graph_objects as go
from rtmha.elevenband import ElevenBand, Wdrc11
from IPython.display import Image, display

In [ ]:
def plot_up (title, inp, eb, num, name=None):
    output = eb.get_band(num)
    fig = go.Figure()

    # Create x values. 32 samples per millisecond
    x = np.array(range(len(inp)))/32.0

    fig.add_trace(go.Scatter(x=x,
                    y=inp,
                    opacity=.5,
                    mode='lines+markers',
                    name='input'))

    fig.add_trace(go.Scatter(x=x,
                    y=output,
                    opacity=.5,
                    mode='lines+markers',
                    name='output'))

    fig.update_layout(title=title,
                    xaxis_title='Time(ms)',
                    yaxis_title='Output',
                    template='plotly_dark')
    if name is None:
        fig.show()        
    else:
        fig.write_image(name, engine="kaleido")
        display(Image(name, width=4096))

In [ ]:
eb = ElevenBand(128, True, True)
for num, f in enumerate(eb.freqs):
    _, inp = generate_sine_waves([f], .1)
    eb = ElevenBand(len(inp), True, True)
    eb.create_bands(inp)
    eb.upsample()
    plot_up(f"Downsampled and Upsampled {f}Hz", inp, eb, num, f"{f}out.png")

In [ ]:
g50 = np.ones(11) * 0
g80 = np.ones(11) * 0
kneelow = np.ones(11) * 45
band_mpo = np.ones(11) * 110
AT = np.ones(11) * 10
RT = np.ones(11) * 100
cal_mic = np.ones(11) * 0
cal_rec = np.ones(11) * 0
min_phase=1
align=1

In [ ]:
w = Wdrc11(g50, g80, kneelow, band_mpo, AT, RT, cal_mic, cal_rec, len(inp), min_phase, align)
w.get_param()

In [ ]:
g50[0]=10
g80[0]=10

In [ ]:
for num, f in enumerate(eb.freqs):
    _, inp = generate_sine_waves([f], .1)
    g50 = np.ones(11) * 0
    g80 = np.ones(11) * 0
    g50[num] = 5
    g80[num] = 5
    w = Wdrc11(g50, g80, kneelow, band_mpo, AT, RT, cal_mic, cal_rec, len(inp), min_phase, align)
    w.create_bands(inp)
    w.magnitude()
    mag = w.get_mag(num)
    mag_log = 20 * np.log10(mag) + 96.9897000434
    print(mag_log)
    w.agc()
    w.compress()
    w.upsample()
    plot_up(f"Downsampled and Upsampled {f}Hz", inp, w, num, f"{f}out.png")